#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계2 : 전처리

## 0.미션

* 1) 데이터 분할
    * 학습/평가 데이터를 분리합니다.

* 2) 전처리
    * 학습을 위한 전처리 : 자연어 처리를 위한 전처리를 수행합니다.
        * 맞춤법 검사 라이브러리를 활용하여 올바른 문장으로 변환합니다.
    * 전처리 결과를 저장합니다.

## 1.환경준비

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.
    - 구글드라이브를 통해 데이터를 로딩합니다.

### (1) 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np

import joblib

import os

import re

### (2) 데이터 불러오기

* 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차


In [ ]:
data = joblib.load('data')
data

,intent,Q,A,type
0,1,떨어뜨려서 핸드폰 액정 나갔어,as 맡기세요.,0
1,1,액정 나갔어,as 맡기세요.,0
2,1,핸드폰 떨어뜨려서 고장 났나봐,as 맡기세요.,0
3,1,노트북이 작동이 안되네,AS센터에 맡겨보세요.,0
4,1,노트북 키보드가 안먹히네,AS센터에 맡겨보세요.,0
...,...,...,...,...
787,53,KT 에이블스쿨만의 장점이 무엇인지 상세히 설명해주세요,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
788,53,다른 교육과의 차이점은 무엇인가요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
789,53,다른 교육과 차이점이 있나요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
790,53,다른 국비 지원 교육보다 특별한 점이 있는지 궁금해요,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1


### (1) 학습/평가 데이터 분리
   * test 데이터 : intent마다 무작위로 질문을 뽑아 test 데이터로 분리
   * train 데이터 : 나머지 데이터

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True, stratify=data['intent'])
val, test = train_test_split(test, test_size=0.5, random_state=42, shuffle=True, stratify=test['intent'])

In [ ]:
len(train), len(val), len(test)

(1038, 130, 130)

In [ ]:
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
train

,intent,Q,A,type
0,43,서울 외에도 교육 장소가 있나요?,"KT 에이블스쿨은 전국 어디서나 참여할 수 있도록, 온/오프라인을 병행하여 탄력적으...",1
1,32,고용보험에 가입되어 있지만 않으면 미취업자로 보나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1
2,53,다른 국비지원과 차이점이 있나요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1
3,20,회식 했으면 좋겠다.,맛있는거 드세요.,0
4,9,우울증 생겼어,꾸준히 약 먹고 치료해보세요.,0
...,...,...,...,...
1033,52,대면 교육 때 머무를 수 있는 곳이 있나요?,숙소 및 기숙사에 대한 지원은 제공하지 않습니다.,1
1034,35,결과발표 이후에 추가합격이 있나요?,현재 KT 에이블스쿨은 추가합격 제도를 운영하고 있지 않습니다.\n향후 추가합격을 ...,1
1035,42,교육 수료 시 출석율이 반영되나요?,KT 에이블스쿨은 고용노동부 K-Digital Training 사업으로 운영됨에 따...,1
1036,26,번호 따볼까나,용기를 내보세요!,0


## 2.데이터 전처리

* 세부요구사항
    * 다음 항목에 해당되는 전처리 함수를 각각 생성하여 하나로 합쳐봅시다.
        * 특수문자 제거 : 정규 표현식을 활용합니다.
        * 형태소 분석기 사용 : konlpy(mecab, kkma, Okt, etc), soynlp, khaiii 등 여러 형태소 분석기를 활용합니다.
    * 문장길이를 조절하여 모델 input으로 활용하기
    * joblib.dump를 이용하여 전처리 결과 데이터를 저장합니다.

### (1) 맞춤법 검사하기
   * 웹 스크래핑 - Daum 사전 맞춤법 검사  
       🔗 https://dic.daum.net/grammar_checker.do  
       🔗 https://velog.io/@codnjs042/웹-스크래핑을-통해-Daum-사전-맞춤법-검사-이용하기
       * 함수를 별도 파이썬 파일로 저장

In [ ]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup

# 맞춤법 검사 함수
def spell_check(text):
    url = "https://dic.daum.net/grammar_checker.do"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    data = {
        "sentence": text
    }

	# 맞춤법 검사 요청
    response = requests.post(url, headers=headers, data=data)

    # 맞춤법 검사 응답
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        result_div = soup.find("div", {"class": "cont_grammar"})

        # 맞춤법 오류 수정
        corrected_spell = result_div.find_all("a", {"class": "txt_word txt_spell_high"})
        for span in corrected_spell:
            error_input = span['data-error-input']
            error_output = span['data-error-output']
            text = text.replace(error_input, error_output)

        # 띄어쓰기 오류 수정
        corrected_space = result_div.find_all("a", {"class": "txt_spell txt_spell_high"})
        for span in corrected_space:
            error_input = span['data-error-input']
            error_output = span['data-error-output']
            text = text.replace(error_input, error_output)

        # 맞춤법 검사 결과
        return text

    # 에러 처리
    else:
        return f"Error: {response.status_code}"

# 검사할 텍스트
text = "우리 개곡 놀러가자"
corrected_text = spell_check(text)

print("원본 텍스트:", text)
print("맞춤법 및 띄어쓰기 검사 후 텍스트:", corrected_text)

원본 텍스트: 우리 개곡 놀러가자
맞춤법 및 띄어쓰기 검사 후 텍스트: 우리 계곡 놀러 가자


In [ ]:
train['check'] = train['Q'].apply(lambda x : spell_check(x))
val['check'] = val['Q'].apply(lambda x : spell_check(x))
test['check'] = test['Q'].apply(lambda x : spell_check(x))

In [ ]:
train

,intent,Q,A,type,check
0,43,서울 외에도 교육 장소가 있나요?,"KT 에이블스쿨은 전국 어디서나 참여할 수 있도록, 온/오프라인을 병행하여 탄력적으...",1,서울 외에도 교육 장소가 있나요?
1,32,고용보험에 가입되어 있지만 않으면 미취업자로 보나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1,고용보험에 가입되어 있지만 않으면 미취업자로 보나요?
2,53,다른 국비지원과 차이점이 있나요?,KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무...,1,다른 국비지원과 차이점이 있나요?
3,20,회식 했으면 좋겠다.,맛있는거 드세요.,0,회식했으면 좋겠다.
4,9,우울증 생겼어,꾸준히 약 먹고 치료해보세요.,0,우울증 생겼어
...,...,...,...,...,...
1033,52,대면 교육 때 머무를 수 있는 곳이 있나요?,숙소 및 기숙사에 대한 지원은 제공하지 않습니다.,1,대면 교육 때 머무를 수 있는 곳이 있나요?
1034,35,결과발표 이후에 추가합격이 있나요?,현재 KT 에이블스쿨은 추가합격 제도를 운영하고 있지 않습니다.\n향후 추가합격을 ...,1,결과발표 이후에 추가합격이 있나요?
1035,42,교육 수료 시 출석율이 반영되나요?,KT 에이블스쿨은 고용노동부 K-Digital Training 사업으로 운영됨에 따...,1,교육 수료 시 출석률이 반영되나요?
1036,26,번호 따볼까나,용기를 내보세요!,0,번호 따볼까나


### (5) 전처리 완료된 학습/평가 데이터를 파일로 저장하기
- 총 3개의 파일을 저장합니다.
    * train 셋
    * val 셋
    * test 셋
- 저장방법
    * joblib 라이브러리를 이용하여 저장합니다.

In [ ]:
joblib.dump(train, 'train')
joblib.dump(val, 'val')
joblib.dump(test, 'test')

['test']